Standard dp problem

In [97]:
import numpy as np
import matplotlib.pyplot as plt
import numpy.matlib
%matplotlib inline
import quantecon as qe
import scipy.sparse as sparse
from quantecon import compute_fixed_point
from quantecon.markov import DiscreteDP

In [41]:
# Define a random model I found on the internet
class model:

    def __init__(self, B=10, M=5, α=0.5, β=0.9):
        """
        R = reward array (n x m)
        Q = trnasition probability array (n x m x n)
        β = discount factor [0,1)
        α = quanticy to store and the action
        c = consume (s - α)
        M = global bound of storage
        u(c) = flow utility (c^α)
        s' = next periods stock (a + U)
        U = output, uniform distribution {0,...,B}
        B = maximum output
        S = state space {0,...,M+B}
        n = number of states (M + B + 1) since 0 to M + B
        A(s) = set of feasible actions {0,...,min{s,M}}
        r(s,α) = reward function (u(s - α))
        Q(s,α,s') = transition probability (equation 4.3)
        """

        self.B, self.M, self.α, self.β  = B, M, α, β
        self.n = B + M + 1
        self.m = M + 1

        self.R = np.empty((self.n, self.m))
        self.Q = np.zeros((self.n, self.m, self.n))

        self.populate_Q() 
        self.populate_R()

    def u(self, c):
        return c**self.α

    def populate_R(self):
        """
        Populate the R matrix, with R[s, a] = -np.inf for infeasible
        state-action pairs.
        """
        for s in range(self.n):
            for a in range(self.m):
                self.R[s, a] = self.u(s - a) if a <= s else -np.inf

    def populate_Q(self):
        """
        Populate the Q matrix by setting

            Q[s, a, s'] = 1 / (1 + B) if a <= s' <= a + B

        and zero otherwise.
        """

        for a in range(self.m):
            self.Q[:, a, a:(a + self.B + 1)] = 1.0 / (self.B + 1)

In [522]:
# Define the leapfrogging model
class lf:

    def __init__(self, Cmax = 5, Cmin = 0, nC = 4, pf = 1, k0 = 0, k1 = 8.3, k2 = 1, R = 0.05, Dt = 1):
        """
        Descibe the variables here :(
        all the standard fixed parameters
        """

        self.Cmax, self.Cmin, self.nC, self.pf, self.k0, self.k1, self.k2, self.R, self.Dt  = Cmax, Cmin, nC, pf, k0, k1, k2, R, Dt
        """
        Define the Model parameters
        """
        
        self.p = np.ones(self.nC) # transition probability 
        self.populate_p() # ppupalate with the probabilities
        self.q = 1 - self.p
        self.C = np.linspace(self.Cmax,self.Cmin,self.nC)
        self.beta = np.exp(-self.R*self.Dt)
        self.T = (self.nC-1)*3 + 1
        self.nESS = self.nC * (self.nC + 1) * (2*self.nC+1)/6

        self.firm1 = np.empty((self.nC,self.nC,self.nC)) # initialize the two firms with empty values 
        self.firm1[:] = np.nan
        self.firm2 = self.firm1.copy()
        self.populate_firms()
        
        self.stage_index = self.rlsindex(self.nC)
        print(self.stage_index)

        # Functional forms
        """
        Lambda functions to be defined:
        K
        Payoff
        r1
        r2
        Phi
        """
        self.K = @(c) mp.k0+mp.k1/(1+mp.k2*c)

    def rlsindex(self, nC):
        Ns = 0
        for i in range(0,nC+1):
            Ns = Ns + i**2
        T = 1 + 3 * (nC-1)
        Ns_in_stages = np.ones((1,T))
        j = 0
        l = nC
        while l>1:
            Ns_in_stages[0,j] = 1
            j += 1
            Ns_in_stages[0,j] = 2*(l-1)
            j += 1
            Ns_in_stages[0,j] = (l-1)**2
            j += 1
            l -= 1
        return np.cumsum(Ns_in_stages)

    def populate_p(self):
        """
        Simple list with 0 as last value
        """

        for a in range(0,self.nC):
            if a < self.nC - 1:
                self.p[a] = self.p[a]*self.pf
            else:
                self.p[a] = 0

    def populate_firms(self):
        for i in range(0,self.nC):
            self.firm2[i,:i+1,:i+1] = np.matlib.repmat(C[:i+1],i+1,1)
            self.firm1[i] = self.firm2[i].transpose().copy()
        

In [523]:
testlf = lf()

[ 1.  7. 16. 17. 21. 25. 26. 28. 29. 30.]


In [510]:
test = np.ones((1,10))

In [516]:
test[0,0]

1.0